# Test the in-cloud access of TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2

![image](https://podaac.jpl.nasa.gov/Podaac/thumbnails/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2.jpg)


More information of the data used in this demo can be found on the [podaac landing page](https://doi.org/10.5067/TEMSC-3JC62)

Drafted by Munish Sikka.

Revised by Jinbo Wang.

In [ ]:
%%capture
import requests
import numpy as np
import time
import xarray as xr
import pylab as plt


def store_aws_keys(endpoint: str="https://archive.podaac.earthdata.nasa.gov/s3credentials"):    
    with requests.get(endpoint, "w") as r:
        accessKeyId, secretAccessKey, sessionToken, expiration = list(r.json().values())

    creds ={}
    creds['AccessKeyId'] = accessKeyId
    creds['SecretAccessKey'] = secretAccessKey
    creds['SessionToken'] = sessionToken
    creds['expiration'] = expiration
    
    return creds


creds = store_aws_keys()
print(creds)

In [ ]:
import s3fs
s3 = s3fs.S3FileSystem(
    key=creds['AccessKeyId'],
    secret=creds['SecretAccessKey'],
    token=creds['SessionToken'],
    client_kwargs={'region_name':'us-west-2'},
)

In [ ]:
print(f"\nThe current session token expires at {creds['expiration']}.\n")

In [ ]:
# PO.DAAC's 'short name' is an identifier for the dataset
ShortName = 'TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2'

In [ ]:
# Ask PODAAC for the collection id using the 'short name'
response = requests.get(
    url='https://cmr.earthdata.nasa.gov/search/collections.umm_json', 
    params={'provider': "POCLOUD",
            'ShortName': ShortName,
            'page_size': 1}
)

ummc = response.json()['items'][0]
ccid = ummc['meta']['concept-id']
print(f'collection id: {ccid}')

In [ ]:


## this GRACE dataset provided as only one file spanning the period 2002 to 2022
## unlike other datasets hosted by PO.DAAC which tend to be provided 
## as one file per time level (e.g., month or year).  

# GRCTellus.JPL.200204_202201.GLO.RL06M.MSCNv02CRI.nc

start_time = time.time()

#ss = "podaac-ops-cumulus-protected/" + ShortName + '/GRCTellus.JPL.200204_202201.GLO.RL06M.MSCNv02CRI.nc'

ss="podaac-ops-cumulus-protected/%s/*.nc"%ShortName
GRACE_s3_files = np.sort(s3.glob(ss))
print(GRACE_s3_files)


In [ ]:
data=xr.open_dataset(s3.open('podaac-ops-cumulus-protected/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2/GRCTellus.JPL.200204_202201.GLO.RL06M.MSCNv02CRI.nc'))

In [ ]:
data

## Plot an snapshot

In [ ]:
fig=plt.figure(figsize=(16,10))
data['lwe_thickness'][0,...].plot(vmin=-100,vmax=100,cmap=plt.cm.bwr)

## Plot the temporal variability

In [ ]:
fig=plt.figure(figsize=(16,10))
data['lwe_thickness'].var(axis=0).plot(vmin=-100,vmax=100,cmap=plt.cm.bwr)